In [1]:
import streamlit as st
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams, PayloadSchemaType
from qdrant_client.http import exceptions as qdrant_exceptions

# CONFIG: qdrant
api_key = st.secrets["QDRANT_API_KEY"]
url = st.secrets["QDRANT_URL"]  # for cloud

qdrant_collection_name = "ASK_vectorstore"
# for local instance /private/tmp/local_qdrant
qdrant_path = "/Users/drew_wilkins/Drews_Files/Drew/Python/VSCode/ASK/data/qdrant"

In [11]:
client = QdrantClient(
    url=url,
    api_key=api_key,
)

try:
    '''usage'''
    content = client.get_collections()
    print(content)
except qdrant_exceptions.UnexpectedResponse as e:
    # Check if the error is a 404 Not Found
    if "404" in str(e):
        print("The server returned a 404 Not Found error, which indicates the server is active but could not find the requested URL or endpoint. This might be due to a wrong URL, an incorrect path, or a resource that doesn't exist.")
    else:
        # Re-raise the error if it's not a 404
        raise
except Exception as e:
    # Handle any other exceptions that may occur
    print(f"An unexpected error occurred: {e}")

collections=[CollectionDescription(name='ask_pdf_docs'), CollectionDescription(name='ASK_vectorstore')]


### Create the PDF Document collection

In [6]:
pdf_document_vectors_config = VectorParams(size=1536, distance=Distance.COSINE)

# Create the PDF_document collection
client.create_collection(
    collection_name="ask_pdf_docs",
    vectors_config=pdf_document_vectors_config,
    shard_number=1,  # Adjust based on your sharding needs
    replication_factor=1,  # Adjust if you need replication in a distributed setup
    on_disk_payload=True,
)  # Store payload on disk to save RAM

True

In [12]:
# Define payload schema fields that you want to index for fast filtering
document_fields_to_index = [
    ("title", PayloadSchemaType.TEXT),
    ("leadership_scope", PayloadSchemaType.TEXT),
    # Use timestamp format if necessary
    ("creation_date", PayloadSchemaType.DATETIME),
    ("effective_date", PayloadSchemaType.DATETIME),
    ("upsert_date", PayloadSchemaType.DATETIME),
    ("expiration_date", PayloadSchemaType.DATETIME),
    ("aux_specific", PayloadSchemaType.BOOL),
    ("public_release", PayloadSchemaType.BOOL),
    ("publication_number", PayloadSchemaType.TEXT),
    ("source", PayloadSchemaType.TEXT),
    ("organization", PayloadSchemaType.TEXT),
    ("curator", PayloadSchemaType.TEXT),
    ("file_name", PayloadSchemaType.TEXT),
]

# Create indexes for relevant fields
for field_name, field_type in document_fields_to_index:
    client.create_payload_index(
        collection_name="ask_pdf_docs",
        field_name=field_name,
        field_schema=field_type
    )

### Create the PDF Page collection

In [ ]:
pdf_page_vectors_config = VectorParams(size=1536, distance=Distance.COSINE)

# Create the PDF_document_page collection
client.create_collection(
    collection_name="ask_pdf_pages",
    vectors_config=pdf_page_vectors_config,
    shard_number=1,  # Adjust based on your needs
    on_disk_payload=True  # Store payload on disk to save RAM
)

In [24]:

# Define payload schema fields to index for fast filtering
page_fields_to_index = [
    ("content", PayloadSchemaType.TEXT),  # Content of the page
    ("title", PayloadSchemaType.TEXT),  # Linked from the PDF_document
    ("page_number", PayloadSchemaType.INTEGER),
    ("publication_number", PayloadSchemaType.TEXT),
    ("pdf_doc_id", PayloadSchemaType.UUID),
]

# Create indexes for relevant fields
for field_name, field_type in page_fields_to_index:
    client.create_payload_index(
        collection_name="ask_pdf_pages",
        field_name=field_name,
        field_schema=field_type,
    )

UnexpectedResponse: Unexpected Response: 400 (Bad Request)
Raw response content:
b'{"status":{"error":"Format error in JSON body: data did not match any variant of untagged enum PayloadFieldSchema at line 1 column 49"},"time":0.0}'

In [ ]:
client.create_collection(
    collection_name="pdf_pages",
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE),
    shard_number=1,  
    on_disk_payload=True  
    
    
page_fields_to_index = [
    ("content", PayloadSchemaType.TEXT),  
    ("title", PayloadSchemaType.TEXT),  
    ("page_number", PayloadSchemaType.INTEGER),
    ("parent_pdf_id", PayloadSchemaType.UUID), # adding this line results in an error
]

# Create indexes for relevant fields
for field_name, field_type in page_fields_to_index:
    client.create_payload_index(
        collection_name="pdf_pages",
        field_name=field_name,
        field_schema=field_type,
    )